# **Загрузка данных и Import библиотек**

In [1]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 16057, done.
remote: Total 16057 (delta 0), reused 0 (delta 0), pack-reused 16057
Receiving objects: 100% (16057/16057), 14.68 MiB | 16.68 MiB/s, done.
Resolving deltas: 100% (11032/11032), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.3/645.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.1.0 which is incompatible.


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!git clone https://github.com/academy21/TC-Satellite-DataSet/

Cloning into 'TC-Satellite-DataSet'...
remote: Enumerating objects: 26872, done.
remote: Counting objects: 100% (17389/17389), done.
remote: Compressing objects: 100% (8392/8392), done.
remote: Total 26872 (delta 7), reused 17373 (delta 2), pack-reused 9483
Receiving objects: 100% (26872/26872), 8.02 GiB | 16.12 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Updating files: 100% (27134/27134), done.


In [10]:
from glob import glob
import shutil
import torch
import utils
import cv2
from sklearn.metrics import mean_squared_error as mse
from PIL import Image
import random
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np
display = utils.notebook_init()

YOLOv5 🚀 v7.0-240-g84ec8b5 Python-3.10.12 torch-2.1.0+cu118 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.8/107.7 GB disk)


In [28]:
#!rm -rf /content/drive/MyDrive/AI_Sirius_project_Cyclon_detection/train

In [29]:
#!rm -rf /content/drive/MyDrive/AI_Sirius_project_Cyclon_detection/valid

# **Data Processing**

In [7]:
img = cv2.imread('/content/yolov5/TC-Satellite-DataSet/TC_by_Classes_jpg_1/mt201704240800_ir1_pro.jpg')

In [8]:
img_width = img.shape[0]
img_hight = img.shape[1]

In [9]:
transform = {
    'Class_1' : 0,
    'Class_2' : 1,
    'Class_3' : 2,
    'Class_4' : 3,
    'Class_5' : 4,
    'Class_6' : 5,
    'Class_7' : 6,
}

In [10]:
def transform_label_to_yolov_type(path):
  file = open(path, 'r')
  a = [i.split(',') for i in file.readlines()]
  file = open(path, 'w')
  for line in a:
    Class, x, y, w, h = line
    x, y, w, h = map(int, [x, y, w, h])
    l_x = (x - w / 2) / img_width #  координаты левого верхнего угла
    l_y = (y + h / 2) / img_hight
    r_x = (x + w / 2) / img_width # координаты правого нижнего угла
    r_y = (y - h / 2) / img_hight
    file.write(f'{transform[Class]} {l_x} {l_y} {r_x} {r_y}' + '\n')
def transform_data_to_yolov_type(txt_directory):
  pathes = glob(txt_directory + '/*')
  for path in pathes:
    transform_label_to_yolov_type(path)

In [11]:
transform_data_to_yolov_type('/content/yolov5/TC-Satellite-DataSet/TC_by_Classes_txt_1')
transform_data_to_yolov_type('/content/yolov5/TC-Satellite-DataSet/TC_by_Classes_txt_2')

In [ ]:
!mkdir '/content/yolov5/labels/'
!mkdir '/content/yolov5/images/'

In [13]:
pathes = glob('/content/yolov5/TC-Satellite-DataSet/TC_by_Classes_txt_2/*') + glob('/content/yolov5/TC-Satellite-DataSet/TC_by_Classes_txt_1/*')
for path in pathes:
  path_to_folder = '/content/yolov5/labels/'
  path_to_file = path
  shutil.copy(path_to_file, path_to_folder)

In [14]:
pathes = []
for i in range(1, 5 + 1):
    pathes += glob(f'/content/yolov5/TC-Satellite-DataSet/TC_by_Classes_jpg_{i}/*')
for path in pathes:
  path_to_folder = '/content/yolov5/images/'
  path_to_file = path
  shutil.copy(path_to_file, path_to_folder)

In [12]:
len(glob('/content/drive/MyDrive/AI_Sirius_project_Cyclon_detection/train/images/*'))

3000

In [17]:
import os
pathes = glob('/content/yolov5/images/*')
for path in pathes:
  os.rename(path, path[:-8]+'.jpg')

In [4]:
import random

In [30]:
pathes = glob('/content/yolov5/labels/*')
random.shuffle(pathes)
n = len(pathes)
train_pathes = pathes[:5000]
valid_pathes = pathes[7000:8000]

In [31]:
train_pathes[0]

'/content/yolov5/labels/mt201803302000_ir1.txt'

In [ ]:
# !rm -rf /content/drive/MyDrive/AI_Cyclon_detection_Sirius/train

In [ ]:
# !rm -rf /content/drive/MyDrive/AI_Cyclon_detection_Sirius/valid

## **Moving Data to Drive**

In [32]:
my_train_path = '/content/drive/MyDrive/AI_Sirius_project_Cyclon_detection/train' # укажите путь к папке train на диске
for path in train_pathes:
    path_to_folder = f'{my_train_path}/labels/'
    path_to_file = path
    shutil.copy(path_to_file, path_to_folder)
    path_to_folder = '/content/drive/MyDrive/AI_Sirius_project_Cyclon_detection/train/images/'
    path_to_file = (path.replace('labels', 'images')).replace('txt', 'jpg')
    shutil.copy(path_to_file, path_to_folder)

In [33]:
my_valid_path = '/content/drive/MyDrive/AI_Sirius_project_Cyclon_detection/valid' # укажите путь к папке valid на диске
for path in valid_pathes:
    path_to_folder = f'{my_valid_path}/labels/'
    path_to_file = path
    shutil.copy(path_to_file, path_to_folder)
    path_to_folder = '/content/drive/MyDrive/AI_Sirius_project_Cyclon_detection/valid/images/'
    path_to_file = (path.replace('labels', 'images')).replace('txt', 'jpg')
    shutil.copy(path_to_file, path_to_folder)

# **Train Yolov5**

In [ ]:
!python train.py --img 640 --batch 64 --epochs 3 --data data.yaml --weights yolov5n.pt --hyp data/hyps/hyp.scratch-low.yaml